In [2]:
import pandas as pd
import scipy as sp
from scipy.optimize import curve_fit
from datetime import *
import plotly.express as px
import plotly.graph_objects as go
import scipy.special as spec
from plotly.subplots import make_subplots
import os
import plotly
DATA_OWID = pd.read_csv('https://github.com/owid/covid-19-data/blob/master/public/data/owid-covid-data.csv?raw=true')
DATA_GOOGLE = pd.read_csv("https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv",low_memory=False)

In [52]:
def smoooooth(alpha, y):
    num_els = len(y)
    y_smoothed = y 
    for i in range(1, num_els):
        if y[i-1] > 0.0 or y[i-1] <0.0:
            y_smoothed[i] = alpha * y[i] + (1.-alpha) *y[i-1]
    return y_smoothed
# approximant
def fit_fun(x,A,x0,k):
    #better use this to avoid overflows becasue of using exp
    return A*spec.expit(k*(x - x0))
    #return A / (1 + sp.exp(-k*(x-x0))) 

def dfit_fun(x,A,x0,k):
    return A* sp.exp(-k*(x-x0)) * k / (1 + sp.exp(-k*(x-x0)))**2

def date(string, format="%Y-%m-%d"):
    return datetime.strptime(string, format)

# normalization
"""
def normalize(df):
    df.loc[:, "new_cases_norm"] = (df.loc[:, "new_cases"]/df.loc[:, "new_tests"]).values
    df.loc[:, "total_cases_norm"] = (df.loc[:, "new_cases_norm"]).values
    for i in df.index[1:]:
        df.loc[i, "total_cases_norm"] = df.loc[i-1, "total_cases_norm"] + df.loc[i, "new_cases_norm"]
"""

def get_increase(total_cases):
    prev_day = sp.concatenate((sp.array([0]),total_cases.iloc[:-1].values))
    return pd.Series(total_cases.values - prev_day)

def set_str_to_zero(array):
    ret = sp.zeros(len(array), dtype=float)
    for i in range(len(array)):
        if type(array[i]) == str:
            ret[i] = 0.0
        else:
            ret[i] = array[i]
    return ret

# loads data from Our World in Data for country specified by iso code (3 letter)
# returns a DataFrame
def load_data(code):
    df = DATA_OWID.loc[DATA_OWID["iso_code"] == code,:].reset_index(drop=True)
    df.loc[:,"date"] = pd.to_datetime(df.loc[:,"date"], format="%Y-%m-%d")
    first_day_cases = df.loc[0,"total_cases"]
    first_day = 0
    while first_day_cases == 0 :
        first_day += 1
        first_day_cases = df.loc[first_day, "total_cases"]
    if first_day > 0:
        df = df.iloc[first_day-1:]
    
    return df.reset_index(drop=True)

# loads data for country from file (csv)
# location - as indicated in file (if given)
# *_col: names of corresponding columns in file
def load_data_generic(file, date_col, tot_cases_col, loc_col=None, location = None, date_format="%Y-%m-%d"):
    data = pd.read_csv(file)
    if loc_col is None:
        df = data.loc[:, [date_col, tot_cases_col]]
        
        if location is None:            
            df.insert(0, "location", "Location is not specified")
        else:
            df.insert(0, "location", location)
    else:
        df = data.loc[:, [loc_col, date_col, tot_cases_col]]
        if not location is None:
            df = df.loc[df[loc_col] == location]
        df.rename(columns={loc_col: "location"})
    display(df)
    df = df.rename(columns={date_col: "date", tot_cases_col: "total_cases"})
    df.loc[:,"date"] = pd.to_datetime(df["date"], format=date_format)
    df["new_cases"] = get_increase(df["total_cases"])
    first_day_cases = df["total_cases"].iloc[0]
    first_day = 0
    while int(first_day_cases) == 0 :
        first_day += 1
        first_day_cases = df["total_cases"].iloc[first_day]
    if first_day > 0:
        df = df.iloc[first_day-1:]
    return df.reset_index(drop=True)

# load mobility data from google
# code - iso 2 code
def load_mobility_data(code, sub_reg_1 = None, sub_reg_2 = None):
    df = DATA_GOOGLE
    df = df.loc[df.loc[:,"country_region_code"]==code]
    if sub_reg_1 == None:
        sr = set_str_to_zero(df.loc[:,"sub_region_1"].values)
        df = df.loc[sp.isnan(sr)]
    elif not sub_reg_1 == "All":
        df = df.loc[df.loc[:,"sub_region_1"] == sub_reg_1]
    if sub_reg_2 == None:
        sr = set_str_to_zero(df.loc[:,"sub_region_2"].values)
        df = df.loc[sp.isnan(sr)]
    elif not sub_reg_2 == "All": 
        df = df.loc[df.loc[:,"sub_region_2"] == sub_reg_2]
    df.loc[:,"date"] = pd.to_datetime(df.loc[:,"date"], format="%Y-%m-%d")
    return df.reset_index(drop=True)

# approximation
# df is the dataframe from load_data
# high_cutoff_date sets the last date for fitting, if given 
# low_cutoff_date sets the first date for fitting, if given 
# format by default is e.g. "2020-03-27"
# retruns optimal parameters of fit_fun and covariance matrix
def fit(df, high_cutoff_date=None, low_cutoff_date=None):
    df_cut = df
    if high_cutoff_date is not None:
        df_cut = df_cut.loc[df_cut.loc[:,"date"] <= high_cutoff_date]
    if low_cutoff_date is not None:
        df_cut = df_cut.loc[df_cut.loc[:,"date"] >= low_cutoff_date]
    x = df_cut.index.values
    y = df_cut.loc[:, "total_cases"].values
    y_max = y[-1]
    #print(x, y_norm)
    popt, pcov = curve_fit(fit_fun, x, y, p0 = (y_max, 60, 1))
    return popt, pcov

def get_correlation(x, y):
    return sp.corrcoef(x, y)[0,1]

def get_corr_over_time(df, corr_period = 7, min_corr_days=5):
    data = df.loc[:, ["date", "new_cases", "new_tests"]]
    N_days = len(data)
    df_to_return = pd.DataFrame()
    df_to_return["date"] = data["date"].values[corr_period-1:]
    df_to_return["correlation"] = sp.zeros(N_days - corr_period + 1, dtype = float)
    for i in range(corr_period, N_days+1):
        d = data.iloc[i - corr_period: i,:].dropna()
        if len(d) >= min_corr_days:
            df_to_return.loc[i-corr_period,"correlation"] = get_correlation(d["new_tests"].values, d["new_cases"])
        else:
            df_to_return.loc[i-corr_period,"correlation"] = sp.nan
    return df_to_return
    


# generates all posible fits described in events_dict and fit over all data
# events_dict must have keys "date" with the date of an event and "event" with short description
# return DataFrame with popt and pcov
def get_fit_of_cutoffs(df, cutoffs_dict, format="%Y-%m-%d", add_fit_of_all_data = False):
    dates = cutoffs_dict["date"]
    #print(dates)
    descrps = cutoffs_dict["description"]
    popt = []
    pcov = []
    for d in dates:
        po, pc = fit(df, d)
        popt.append(po)
        pcov.append(pc)
    if add_fit_of_all_data:
        po, pc = fit(df)
        popt.append(po)
        pcov.append(pc)
        dates = dates.append(pd.Series(df["date"].iloc[-1]))
        descrps = descrps.append(pd.Series("Last available data"))
    params=pd.DataFrame()
    params["date"] = dates
    params["description"] = descrps
    params["popt"] = popt
    params["pcov"] = pcov
    return params.reset_index()

# plots camulative cases and total over time and approximants, if params given
# plots mobility if df given
# df - data from load_data
# df - data from load_data_mobility
# params - from get_fit_of_events 
def plotter(df, params=None, df_mobility = None ,N_pred = 0, correlation = 0, template = "plotly"):
    # prepare go's
    x_min = min(df["date"].iloc[0], df_mobility["date"].iloc[0])
    
    x_max = max(df["date"].iloc[-1], df_mobility["date"].iloc[-1])
    x_len = x_max - x_min
    x_min = x_min - x_len*0.05
    x_max = x_max + x_len*0.05
    fig_merged = make_subplots(specs=[[{"secondary_y": True}]])
    fig_camulative = make_subplots(specs=[[{"secondary_y": True}]])
    fig_new = make_subplots(specs=[[{"secondary_y": True}]])
    fig_camulative.update_layout(
        title=df.loc[0,"location"],
        xaxis_title="Date",
        yaxis_title="Camulative number of cases"
    )
    fig_new.update_layout(
        title=df.loc[0,"location"],
        xaxis_title="Date",
        yaxis_title="Number of new cases"
    )
    fig_merged.update_layout(
        title=df.loc[0,"location"],
        xaxis_title="Date",
        yaxis_title="Camulative number of cases"
    )
    fig_merged.update_yaxes(title_text="Number of new cases", secondary_y=True)
    figs = [fig_camulative, fig_new, fig_merged]
    # prepare x-axis data
    x = df["date"]
    #prepare y-axis data
    y_camulative = df["total_cases"].values
    y_new = df["new_cases"].values
    # plot actual data
    fig_camulative.add_trace(go.Scatter(x=x, y=y_camulative, mode='markers', name="Data"))
    fig_new.add_trace(go.Scatter(x=x, y=y_new, mode='markers', name="Data"))
    fig_merged.add_trace(go.Scatter(x=x, y=y_camulative, mode='markers', name="Data (camulative)"))
    fig_merged.add_trace(go.Scatter(x=x, y=y_new, mode='markers', name="Data (new cases)"), secondary_y=True)
    # plot approximants
    if not params is None:
        x_pred = pd.date_range(df["date"].iloc[0], df["date"].iloc[-1] + timedelta(days=N_pred+1))  
        x_pred_for_fun = sp.arange(df.index[0], df.index[-1]+N_pred+1)
        for i in range(len(params)):
            y = fit_fun(x_pred_for_fun, *params["popt"].iloc[i])
            dy = dfit_fun(x_pred_for_fun, *params["popt"].iloc[i])
            fig_camulative.add_trace(go.Scatter(x=x_pred, y=y, name=params["description"].iloc[i]))
            fig_new.add_trace(go.Scatter(x=x_pred, y=dy, name=params["description"].iloc[i]))
            fig_merged.add_trace(go.Scatter(x=x_pred, y=y, name=params["description"].iloc[i]))
            fig_merged.add_trace(go.Scatter(x=x_pred, y=dy, name=params["description"].iloc[i]), secondary_y=True)
            date = params["date"].iloc[i]
            for fig in figs:
                fig.add_shape(
                    dict(
                        type="line",
                        x0=date,
                        x1=date,
                        y0=0,
                        y1=1,
                        yref='paper',
                        line=dict(
                            color="Black",
                            width=1,
                            dash="dash"
                        ),
                        opacity=0.6
                    )
                )    
                
    ret = [fig_camulative, fig_new, fig_merged]
    # plot correlations
    if correlation > 0:
        d = get_corr_over_time(df, corr_period=correlation)
        fig_new.add_trace(go.Scatter(x=d["date"], y=d["correlation"], name="%d-days correlation"%correlation, line=dict(width=1, dash="dash")),secondary_y=True)
        #fig_new.add_trace(go.Scatter(x=df["date"], y=df["new_tests"], name="New tests (left axis)"))
        fig_new.update_yaxes(title_text="Correlation", secondary_y=True)
    
    # plot mobility indecies
    
    if not df_mobility is None:
        
        fig_mob = go.Figure()
        fig_mob.update_layout(
            title=df.loc[0,"location"] + "<br> Mobility",
            xaxis_title="Date",
            yaxis_title="Deviation from the baseline, %"
        )
        fig_mob_av = go.Figure()
        fig_mob_av.update_layout(
            title=df.loc[0,"location"] + "<br> Mobility",
            xaxis_title="Date",
            yaxis_title="Deviation from the baseline, %"
        )
        cols = df_mobility.columns[5:]
        x = df_mobility["date"]
        
        # averages
        y_av = -df_mobility.iloc[:, -1].values
        for i in cols[:-1]:
            y_av += df_mobility[i].values
        y_av = y_av / 6.
        y_av_wo_parks_res = sp.zeros(len(df_mobility))
        for i in [0, 1, 3, 4]:
            y_av_wo_parks_res += df_mobility[cols[i]].values
        y_av_wo_parks_res = y_av_wo_parks_res / 4.
        fig_mob.add_trace(go.Scatter(x=x, y=y_av, name="Average index"))
        fig_mob.add_trace(go.Scatter(x=x, y=y_av_wo_parks_res, name="Average index w/o parks and residentials"))
        # source
        for i in cols:
            y = df_mobility[i]
            fig_mob.add_trace(go.Scatter(x=x, y=y, name=i.replace("_", " ")[:-29] ))
        
        ret.append(fig_mob)
        fig_mob_av.add_trace(go.Scatter(x=x, y=y_av_wo_parks_res, name="Average index"))
        #for alp in sp.linspace(0., 1., 10):
        #    fig_mob_av.add_trace(go.Scatter(x=x, y=smoooooth(alp, y_av_wo_parks_res), name="Smoothed, alp="+str(alp)))
        ret.append(fig_mob_av)
    
    for f in ret:
        f.update_layout(legend_orientation="h", 
                        xaxis_range=[x_min, x_max], 
                        legend=dict(y=-0.2),
                        height=600,
                        width=900,
                        showlegend=True,
                        title="",
                        font=dict(
                            family="Courier New, monospace",
                            size=12,
                            color="#7f7f7f"
                        ),
                        template=template
                       )
    return ret
    

# unfolds popt and pcov  from get_fit_of_events
# returns dataframe with each parameter and its standart deviation
def unfold(params):
    sigmas = []
    for pc in params["pcov"]:
        sigmas.append(sp.sqrt(sp.diagonal(pc)))
    df = params.loc[:, ["date", "description"]]
    df["A"] = [params["popt"][k][0] for k in params.index]
    df["\sigma_A"] = sp.array(sigmas)[:,0]
    df["x_0"] = [params["popt"][k][1] for k in params.index]
    df["\sigma_x_0"] = sp.array(sigmas)[:,1]
    df["k"] = [params["popt"][k][2] for k in params.index]
    df["\sigma_k"] = sp.array(sigmas)[:,2]
    return df
    
# functions for post editing figures
# add a vertical line
def add_vline(fig, x, color="Black", width=1, dash="solid", opacity=1, layer = "above"):
    fig.add_shape(
        type = "line",
        xref="x",
        x0=x,
        x1=x,
        y0=0,
        y1=1,
        yref="paper",
        layer=layer,
        line=dict(
            color=color,
            width=width,
            dash=dash
        ),
        opacity = opacity        
    )
# add a filled region
def add_filling(fig, x0, x1, fill_color="Red", opacity=0.3, layer="above"):
    fig.add_shape(
        type="rect",
        xref="x",
        yref="paper",
        x0=x0,
        y0=0,
        x1=x1,
        y1=1,
        fillcolor=fill_color,
        opacity=opacity,
        layer=layer,
        line_width=0
    )

def add_caption(fig, x, text, pos_x=0, pos_y=-20, text_color="Black", bg_color="Pink", br_color="Grey", ar_color="#606060", br_width=1, ar_width=1, opacity=0.7, layer="above", wline=True):

    fig.add_annotation(
        x=x,
        y=1,
        xref="x",
        yref="paper",
        text=text,
        showarrow=True,
        font=dict(
            family="Courier New, monospace",
            size=12,
            color=text_color
            ),
        align="left",
        arrowhead=2,
        arrowsize=1,
        arrowwidth=ar_width,
        arrowcolor=ar_color,
        ax=pos_x,
        ay=pos_y,
        bordercolor=br_color,
        borderwidth=br_width,
        borderpad=4,
        bgcolor=bg_color,
        opacity=0.7
    )
    if wline:
        add_vline(fig, x, color=br_color, width=br_width, layer=layer)

def add_approximant(fig, params, index, mode="new", N_pred=0):
    x  = fig.data[0].x
    x_pred = pd.date_range(x[0], x[-1] + timedelta(days=N_pred+1))  
    x_pred_for_fun = sp.arange(0, len(x_pred))
    if mode == "new":
        y = dfit_fun(x_pred_for_fun, *params["popt"].iloc[index])
        fig.add_trace(go.Scatter(x=x_pred, y=y, name=params["description"].iloc[index]))
    elif mode == "total":
        y = fit_fun(x_pred_for_fun, *params["popt"].iloc[index])
        fig.add_trace(go.Scatter(x=x_pred, y=y, name=params["description"].iloc[index]))
    elif mode == "total+new":
        y = fit_fun(x_pred_for_fun, *params["popt"].iloc[index])
        fig.add_trace(go.Scatter(x=x_pred, y=y, name=params["description"].iloc[index]))
        y = dfit_fun(x_pred_for_fun, *params["popt"].iloc[index])
        fig.add_trace(go.Scatter(x=x_pred, y=y, name=params["description"].iloc[index]), secondary_y=True)
    else:
        raise Exception('mode must be one of: "new","total" or "total+new"')
    date = params["date"].iloc[index]
    fig.add_shape(
        dict(
            type="line",
            x0=date,
            x1=date,
            y0=0,
            y1=1,
            yref='paper',
            line=dict(
                color="Black",
                width=1,
                dash="dash"
            ),
            opacity=0.6
        )
    )    

    
def add_difference(f, fit_ref = -1, fit=1, pos_x =  7, color="Black", width = 1., layer="above", text_color="Black", opacity=0.75):
    n_a = len(f.data)
    margin =timedelta(days=1)
    y_ref = max(f.data[fit_ref].y)
    ind_ref = sp.argmax(f.data[fit_ref].y)
    x_ref = f.data[fit_ref].x[ind_ref] - margin
    i = fit
    y = max(f.data[i].y)
    ind = sp.argmax(f.data[i].y)
    x = f.data[i].x[ind] - margin
    text = "Todo"+str(y_ref/y)
    x_arr = max(x, x_ref) + timedelta(days=pos_x) 
    x1 = x_arr + margin
    f.add_shape(
        type = "line",
        xref="x",
        x0=x,
        x1=x1,
        y0=y,
        y1=y,
        yref="y",
        layer=layer,
        line=dict(
            color=color,
            width=width
        ),
        opacity = opacity        
    )
    f.add_shape(
        type = "line",
        xref="x",
        x0=x_ref,
        x1=x1,
        y0=y_ref,
        y1=y_ref,
        yref="y",
        layer=layer,
        line=dict(
            color=color,
            width=width
        ),
        opacity = opacity        
    )
    f.add_shape(
        type = "line",
        xref="x",
        x0=x_arr,
        x1=x_arr,
        y0=y,
        y1=y_ref,
        yref="y",
        layer=layer,
        line=dict(
            color=color,
            width=width
        ),
        opacity = opacity        
    )
    y_mid =(y + y_ref)/2.
    y_diff = abs(y - y_ref)
    if y > y_ref:
        text = "Pike is increased<br>by %4.2f %%" % ((y/y_ref - 1.)*100.)
        y_max = y
        y_min = y_ref
    else:
        text = "Pike is lowered<br>by %4.2f %%" % ((1. - y/y_ref)*100.)
        y_max = y_ref
        y_min = y
    x_text = x_arr + timedelta(days=pos_x)
    f.add_annotation(x = x_arr,
                    y = y_min,
                    xref = "x", yref = "y",
                    axref = "x", ayref = "y",
                    text = "",
                    showarrow = True,
                    arrowhead=1,
                    arrowsize=2,
                    arrowwidth=width,
                    arrowcolor=color,
                    ax = x_arr,
                    ay = y_mid)
    f.add_annotation(x = x_arr,
                    y = y_max,
                    xref = "x", yref = "y",
                    axref = "x", ayref = "y",
                    text = "",
                    showarrow = True,
                    arrowhead=1,
                    arrowsize=2,
                    arrowwidth=width,
                    arrowcolor=color,
                    ax = x_arr,
                    ay = y_mid)
    f.add_annotation(
                    x = x_text,
                    y = y_mid,
                    ax = x_text,
                    ay = y_mid,
                    xref = "x", yref = "y",
                    axref = "x", ayref = "y",
                    text=text,
                    font=dict(
                              family="Courier New, monospace",
                              size=12,
                              color=text_color
                             ),
                    align="left",
                    bordercolor=color,
                    borderwidth=width,
                    borderpad=4,
                    bgcolor="LightGreen",
                    opacity=opacity
    )


class image_writer:
    def __init__(self, fig, name, ext, mode=True, scale=1.):
        # delete if not anaconda
        plotly.io.orca.config.executable = '/home/egorim/anaconda3/bin/orca'
        # images saved in ./images/name
        if not os.path.exists("images"):
            os.mkdir("images")
        self.fig = fig
        self.name = name
        self.ext = ext
        self.i = 0
        self.scale=scale
        self.mode=mode
    
    def write(self):
        if self.mode:
            if not os.path.exists("images/"+self.name):
                os.mkdir("images/"+self.name)
            self.fig.write_image("./images/"+self.name+"/"+self.name+"_"+str(self.i)+"."+self.ext, scale=self.scale)
            self.i += 1
    def clear_temp_files(self):
        if self.mode:
            if os.path.exists("images/"+self.name):
                os.system("rm -rf images/"+self.name)
                      
    def merge_pdfs(self):
        if self.mode:
            if ext == "pdf":
                os.system("pdftk images/"+self.name+"/* output images/"+self.name+".pdf")
                self.clear_temp_files()
            else:
                #raise Exception("Only pdf's can be merged")
                sys.stderr.write("Only pdf's can be merged")

## South Korea

In [60]:
# load data
df = load_data("KOR")
df_mobility = load_mobility_data("KR") 

In [ ]:
# set cutoffs for approximations
cutoffs = pd.DataFrame({
    "date": [date("2020-03-09"), date("2020-05-07")],
    "description": ["Before 09.03", "Before the second wave"]
})
params = get_fit_of_cutoffs(df, cutoffs)
# plot figures
template = "ggplot2"
figs = plotter(df, params=None, df_mobility=df_mobility, template=template, correlation=14)
# show coeffs
display(unfold(params))

# set extension of output files
ext = "png"
scale = 2.
# set mode of image_writer (if mode: write images)
mode = True
# plot of mobility
f = figs[-1]
iw = image_writer(f, "SK_mobility"+"_"+ext, ext, mode=mode,scale=scale)
# next add elements on bare plot and write image if needed
f.update_layout(xaxis_range = [date("2020-02-01"), date("2020-04-23")])
iw.write()
add_caption(f, date("2020-02-18"), text="Patient 31", pos_x=-50)
iw.write()
add_filling(f, x0=date("2020-02-18"), x1=date("2020-02-23"))
add_caption(f, date("2020-02-20-12", format="%Y-%m-%d-%H"), "Shincheonji Church incident",pos_x= 10, pos_y = -50, wline=False )
iw.write()
add_caption(f, date("2020-02-20"), "Additional measures imposed<br>Tracking of church members", bg_color="LightGreen",pos_x=-120, pos_y=30)
add_filling(f, date("2020-02-20"), date("2020-03-08"), fill_color="Green")
iw.write()
add_caption(f, date("2020-02-23"), "Drop in mobility", bg_color="LightGreen",pos_x= 50 )
iw.write()
add_caption(f, date("2020-02-28"), "Mobility index reaches minimum", bg_color="LightGreen", pos_x = 190, pos_y=60 )
iw.write()
add_caption(f, date("2020-03-08"), "Tracking of church memebers<br>is completed", bg_color="LightGreen", pos_x = 160, pos_y=-30 )
iw.write()
iw.merge_pdfs()

f.show()

#plot of new cases
f = figs[1]
iw = image_writer(f, "SK_plot_"+ext, ext, mode=mode,scale=scale)
f.update_layout(xaxis_range = [date("2020-02-01"), date("2020-04-23")])
add_caption(f, date("2020-02-18"), text="Patient 31", pos_x=-50)
add_filling(f, x0=date("2020-02-18"), x1=date("2020-02-23"))
add_caption(f, date("2020-02-20-12", format="%Y-%m-%d-%H"), "Shincheonji Church incident", pos_x= 10, pos_y = -50, wline=False )
add_caption(f, date("2020-02-20"), "Additional measures imposed<br>Tracking of church members", bg_color="LightGreen",pos_x=-120, pos_y=30)
add_filling(f, date("2020-02-20"), date("2020-03-08"), fill_color="Green")
add_caption(f, date("2020-02-23"), "Drop in mobility", bg_color="LightGreen",pos_x= 50)
add_caption(f, date("2020-02-28"), "Mobility index reaches minimum", bg_color="LightGreen", pos_x = 190, pos_y=60)
add_caption(f, date("2020-03-08"), "Tracking of church memebers<br>is completed", bg_color="LightGreen", pos_x = 160, pos_y=-30)
iw.write()
add_approximant(f, params, 1)
iw.write()
add_filling(f, date("2020-02-28"), date("2020-03-09"), fill_color="Blue")
add_caption(f, date("2020-03-04"), "10-days <i>incubation</i> period", bg_color="LightBlue", pos_x = 160, pos_y=30, wline=False)
iw.write()
add_approximant(f, params, 0)
iw.write()
add_difference(f, fit_ref=-1, fit=-2, pos_x=8)
iw.write()
iw.merge_pdfs()
f.show()

,date,description,A,\sigma_A,x_0,\sigma_x_0,k,\sigma_k
0,2020-03-09,Before 09.03,7858.428405,82.681852,42.757435,0.089387,0.349544,0.006774
1,2020-05-07,Before the second wave,10321.838373,75.021662,46.422966,0.304445,0.172967,0.007863


Only pdf's can be merged

## Italy


In [55]:
df = load_data("ITA")
df_mobility = load_mobility_data("IT")

In [59]:
cutoffs = pd.DataFrame({
    "date": [date("2020-04-02")],
    "description": ["Before 02.04"]
})
params = get_fit_of_cutoffs(df, cutoffs, add_fit_of_all_data = True)
# plot figures

figs = plotter(df, params=None, df_mobility=df_mobility, template=template, correlation=14)
# show coeffs
display(unfold(params))

# set extension of output files
ext = "png"
scale = 2.
# set mode of image_writer (if mode: write images)
mode = True


f = figs[-1]
iw = image_writer(f, "IT_mobility_"+ext, ext, mode=mode,scale=scale)
iw.write()
add_caption(f, date("2020-02-22"), text="Measures imposed in<br>northern regions", bg_color="LightGreen", pos_x=-80, pos_y = 25)
iw.write()
add_caption(f, date("2020-02-26"), text="Drop in mobilitly<br>in northern regions", bg_color="LightGreen", pos_x=-90, pos_y =70)
iw.write()
add_caption(f, date("2020-03-01"), text="Nationwide measures", bg_color="LightGreen", pos_x=-50)
add_filling(f, date("2020-03-01"), date("2020-03-15"))
add_caption(f, date("2020-03-15"), text="15-days delay betwwen nationwide measures<br>and significant drop in mobility", pos_y=-60, pos_x=150)
iw.write()
add_caption(f, date("2020-03-08"), text="Penalties for violation<br>measures introduced", bg_color="LightGreen", pos_y=-60, pos_x=-60)
iw.write()
add_caption(f, date("2020-03-22"), text="Mobility index reaches minimum", bg_color="LightGreen", pos_x=50)
iw.write()
iw.merge_pdfs()
f.show()

f = figs[1]
iw = image_writer(f, "IT_plot_"+ext, ext, mode=mode,scale=scale)
add_caption(f, date("2020-02-22"), text="Measures imposed in<br>northern regions", bg_color="LightGreen", pos_x=-80, pos_y = 25)
add_caption(f, date("2020-02-26"), text="Drop in mobilitly<br>in northern regions", bg_color="LightGreen", pos_x=-90, pos_y =70)
add_caption(f, date("2020-03-01"), text="Nationwide measures", bg_color="LightGreen", pos_x=-50)
add_caption(f, date("2020-03-08"), text="Penalties for violation<br>measures introduced", bg_color="LightGreen", pos_y=-60, pos_x=-60)
add_caption(f, date("2020-03-15"), text="15-days delay betwwen nationwide measures<br>and significant drop in mobility", pos_y=-60, pos_x=150)
add_caption(f, date("2020-03-22"), text="Mobility index reaches minimum", bg_color="LightGreen", pos_x=50)
add_filling(f, date("2020-03-01"), date("2020-03-15"))
iw.write()
add_approximant(f, params, 1)
iw.write()
add_filling(f, date("2020-03-22"), date("2020-04-02"), fill_color="Blue")
add_caption(f, date("2020-03-28"), text="10-days <i>incubation</i> period", bg_color="LightBlue", pos_x=130, pos_y=15, wline=False)
iw.write()
add_approximant(f, params, 0)
iw.write()
add_difference(f, fit_ref=-1, fit=-2,pos_x=13)
iw.write()
iw.merge_pdfs()
f.show()

,date,description,A,\sigma_A,x_0,\sigma_x_0,k,\sigma_k
0,2020-04-02,Before 02.04,132173.662632,1069.373994,54.362351,0.110998,0.182169,0.001666
1,2020-06-02,Last available data,226831.174139,1226.993989,65.175249,0.264471,0.094536,0.001955


Only pdf's can be merged

Only pdf's can be merged

### China (Hubei)